In [136]:
import pandas as pd
import pandas_ta as ta
import datetime
import numpy as np

In [2]:
#Load model
import pickle
filename='.pkl'
with open(filename, 'rb') as file:
    model = pickle.load(file)


In [152]:
def connectToBinance():
  from binance import Client
  return Client('3FqLXkx5UOAX10B5lVRhmDDFap7uFY8evLeOqFqnwVfmkZh9IY1tr1to8oBwNdRG',
                'UUR5wLkgzqXEMQbRzkwxJlwD9iChahjLyYNanzawTCDX1jIxRUzpxVtZvcSyjAi1')

def getCurrentTicker(client):
  price = client.get_ticker(symbol='BTCUSDT')
  closeTime = pd.to_datetime(datetime.datetime.fromtimestamp(price['closeTime']/1000, tz=datetime.timezone.utc))
  highPrice = np.float32(price['highPrice'])
  lowPrice = np.float32(price['lowPrice'])
  closePrice = np.float32(price['lastPrice'])

  return pd.DataFrame({'high':highPrice,'low':lowPrice, 'close' :closePrice}, index=[closeTime.strftime("%Y-%m-%d")])

def reteiveData(candle):
  with open('last34Days.pkl', 'rb') as file:
    last34Days = pickle.load(file)
  pd.concat([last34Days, candle]).iloc[1:].to_pickle('last34Days.pkl')
  return pd.concat([last34Days, candle])

def save34Days(data):
  data.iloc[1:].to_pickle('last34Days.pkl')
  
def getIndicators(data, indicators=['MACD_12_26_9', 'MACDh_12_26_9','RSI_5', 'RSI_10', 'RSI_15']):
  strat = ta.Strategy(name='Strategie 1',
                     description='MACD, RSI',
                     ta=[
                       {'kind':'macd', 'fast':12 , 'slow':26},
                       {'kind':'rsi', 'length':5},
                       {'kind':'rsi', 'length':10},
                       {'kind':'rsi', 'length':15}
                     ])

  data.ta.strategy(strat)
  return data[indicators].iloc[-1]

def getPrediction(indicators):
  filename='./model/MLP689.pkl'
  with open(filename, 'rb') as file:
    model = pickle.load(file)
  return model.predict(np.array(indicators).reshape(1, -1))
  
def sellPapier(amount, file):
  print(f"Une vente de {amount}BTC à été effectuée.\n\n")
  
def buyPapier(amount, file):
  print(f"Un achat de {amount}BTC à été effectué.\n\n")

def waitPapier(file):
  print("Aucune action à été effectué.\n\n")

def paperTrade(prediction, amount):
  file = open('log.txt', 'a')
  file.write(f"{datetime.datetime.now()} - La predicton {'UP' if prediction == 2 else ('DOWN' if prediction == 0 else 'NO_MOVEMENT')} a été émise.\n")
  
  
  if(prediction == 0):
    sellPapier(amount,file)
  if(prediction == 2):
    buyPapier(amount,file)
  if(prediction == 1):
    waitPapier(amount,file)
  
   
  

client = connectToBinance()
candle = getCurrentTicker(client)
data = reteiveData(candle)
save34Days(data)
indicators = getIndicators(data)
prediction = getPrediction(indicators)[0] 
# 0:Down
# 1:no mouvement greater then +/-50$
# 2:Up
import random
prediction = random.randint(0, 2) ##Evidement à effacer
amount = getAmount(client)
paperTrade(prediction, amount)





# if(prediction == 0):
# elif(prediction==1):
# else:
  


/Users/sam/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


0.0

In [151]:
#Reset last34Days
import pickle
with open('testValue.pkl', 'rb') as file:
  testData = pickle.load(file)
testData.to_pickle('last34Days.pkl')

In [153]:
model

SVC()

In [35]:
#Log prediction
import datetime
import random

prediction = random.randint(0, 2)
f = open('log.txt', 'a')
f.write(f"{datetime.datetime.now()} - La predicton {'UP' if prediction == 2 else ('DOWN' if prediction == 0 else 'NO_MOVEMENT')} a été émise.\n")
f.close()